![jupyter](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

**Схема базы данных**

В базе данных 5 таблиц. 

Таблица `books` 
Содержит данные о книгах:
- `book_id` — идентификатор книги - связывает с таблицами `review` и `ratings`
- `author_id` — идентификатор автора - связывает с таблицей `authors`
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя - связывает с таблицей `publishers`


Таблица `authors`
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`
Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Загрузка данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'XXX', # имя пользователя
 'pwd': 'XXX', # пароль
 'host': 'XXX',
 'port': 6432, # порт подключения
 'db': 'XXX'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Books

In [3]:
query = ''' SELECT *
FROM books
LIMIT 5
'''

In [4]:
books = pd.io.sql.read_sql(query, con = engine)

In [5]:
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя

### Ratings

In [6]:
query = ''' SELECT *
FROM ratings
LIMIT 5
'''

In [7]:
ratings = pd.io.sql.read_sql(query, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги

### Authors

In [8]:
query = ''' SELECT *
FROM authors
LIMIT 5
'''

In [9]:
authors = pd.io.sql.read_sql(query, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


- `author_id` — идентификатор автора;
- `author` — имя автора.

### Publishers

In [10]:
query = ''' SELECT *
FROM publishers
LIMIT 5
'''

In [11]:
publishers = pd.io.sql.read_sql(query, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства

### Reviews

In [12]:
query = ''' SELECT *
FROM reviews
LIMIT 5
'''

In [13]:
reviews = pd.io.sql.read_sql(query, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора

На экран были выведены первые строки из каждой таблицы, что позволило оценить их содержание. 

## Книги, изданные после 01.01.2000

In [14]:
query = ''' 
SELECT COUNT(*) as "Всего"
FROM books
WHERE publication_date > '2000-01-01'
'''

In [15]:
after_01012000 = pd.io.sql.read_sql(query, con = engine)
after_01012000

,Всего
0,819


После 1 января 2000 года было издано 819 книг

## Обзоры и средний рейтинг

In [16]:
query = ''' 
SELECT b.book_id,
       b.title as "Книга", 
       COUNT(DISTINCT rv.review_id) as "Обзоров",
       AVG(rt.rating) as "Средняя оценка"
FROM books AS b
LEFT OUTER JOIN reviews AS rv ON b.book_id = rv.book_id
LEFT OUTER JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY COUNT(DISTINCT rv.review_id) DESC, AVG(rt.rating) DESC
'''

In [17]:
avg_book_rating = pd.io.sql.read_sql(query, con = engine)
avg_book_rating.head(10)

,book_id,Книга,Обзоров,Средняя оценка
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


Средняя оценка книги с наибольшим числом обзоров 3,66. Чаще читатели пишут негативный отзыв, а те, кому книга понравилась не всегда уделяют время на обзоры.

## Издательство

In [19]:
query = ''' 
SELECT pb.publisher as "Издательство", 
    COUNT(b.book_id) as "Книг"
FROM books AS b
INNER JOIN publishers AS pb ON pb.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY pb.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1
'''

In [20]:
main_publisher = pd.io.sql.read_sql(query, con = engine)
main_publisher.head()

,Издательство,Книг
0,Penguin Books,42


Первое место среди издательств занимает "Penguin Books", которое выпустило 42 книги толще 50 страниц

## Автор

In [21]:
query = '''SELECT af.author as "Автор", 
    ROUND(f.avg, 2) as "Средняя оценка"
FROM(SELECT d.author_id, AVG(d.avg_rating)  
     FROM
       (SELECT b.book_id,
               b.author_id,
               COUNT(rt.rating_id),
               AVG(rt.rating) AS avg_rating
        FROM books AS b
        INNER JOIN ratings AS rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
        HAVING COUNT(rt.rating_id) >= 50) AS d
     GROUP BY d.author_id
     ORDER BY AVG(d.avg_rating) DESC
     LIMIT 1) AS f
INNER JOIN authors AS af ON f.author_id = af.author_id'''


In [22]:
top_author = pd.io.sql.read_sql(query, con = engine)
top_author

,Автор,Средняя оценка
0,J.K. Rowling/Mary GrandPré,4.28


Среди книг с 50 и более оценками автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré

## Обзоры

In [23]:
query = '''
SELECT ROUND(AVG(i.review_count), 2) as "Cреднее количество обзоров"
FROM
  (SELECT username,
          COUNT(review_id) AS review_count
   FROM reviews
   GROUP BY username) AS i
WHERE i.username IN
    (SELECT username
     FROM ratings
     GROUP BY username
     HAVING COUNT(rating_id) > 50)
'''

In [24]:
avg_reviews = pd.io.sql.read_sql(query, con = engine)
avg_reviews

,Cреднее количество обзоров
0,24.33


Пользователи, поставившие более 50 оценок в среднем оставляют 24,33 обзора

Для формирования предложения стоит учитывать книги:
- выпущенные после 1 января 2000 года ;)
- из списка популярных книг с самой высокой средней оценкой
- выпущенные издательством "Penguin Books"
- написанные J.K. Rowling/Mary GrandPré